In [1]:
print("Hello World")

Hello World


In [2]:
import pandas as pd
import pyodbc

In [3]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [4]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_07_04_ssd_award;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_5888\270502191.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [5]:
df['award_reference'].isna().sum()

172

In [6]:
print(df['funded_amount'].isna().sum())

26609


In [7]:
df.drop(columns=['announced_start_date', 'profiled_amount', 'project_cost', 'termination_date'], inplace=True )

In [8]:
df['application_reference'].isna().sum()

0

In [9]:
ssd_organisation = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_07_04_ssd_organisation;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_5888\3828076946.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ssd_organisation = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_07_04_ssd_organisation;", con)


In [10]:
ssd_organisation = ssd_organisation[['organisation_id', 'ukprn', 'organisation_name', 'organisation_source']]

In [11]:
ssd_organisation[ssd_organisation[['organisation_id', 'ukprn']].duplicated(keep=False)].sort_values(by='organisation_id')
ssd_organisation[['organisation_id', 'ukprn']].duplicated().sum()

274

In [12]:
df['application_source'].unique()

array(['TFS', 'Siebel', 'Both Siebel & OffSystem', 'OffSystem', 'IUK',
       None], dtype=object)

In [13]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_07_04_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_5888\2332307117.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_07_04_ssd_application;", con)


In [14]:
df[['application_reference', 'application_source']].duplicated().sum()

0

In [15]:
df_application.columns

Index(['application_reference', 'award_reference', 'funder', 'decision_owner',
       'application_source', 'administrator', 'opportunity_id',
       'opportunity_name', 'application_category', 'application_status',
       'derived_status', 'decision_date', 'lead_organisation_id',
       'lead_organisation_department_id', 'application_title',
       'amount_applied_for', 'scheme', 'rejection_reason',
       'date_last_received', 'publish_to_gtr', 'application_stage',
       'application_stage_status', 'joint_proposal', 'parent_award',
       'parent_award_reference', 'redact_abstract_on_gtr',
       'organisation_source', 'international_programme'],
      dtype='object')

In [16]:
data_application = df_application[['application_reference', 'application_source', 'application_title', 'decision_owner', 
                                   'application_category', 'lead_organisation_id', 'organisation_source']]

In [17]:
data_application[['application_reference', 'application_source']].duplicated().sum()

0

In [18]:
data_merge = pd.merge(df, data_application, on=['application_reference', 'application_source'], how='left', indicator=True)

In [19]:
data_merge[data_merge['_merge'] == 'left_only']

,application_reference,award_reference,funder,administrator,application_source,start_date,end_date,funded_amount,application_title,decision_owner,application_category,lead_organisation_id,organisation_source,_merge


In [20]:
ssd_organisation[['organisation_id', 'organisation_source']].duplicated().sum()

0

In [21]:
data_w_org = pd.merge(data_merge.drop(columns='_merge'), ssd_organisation, left_on=['lead_organisation_id', 'organisation_source'],
         right_on=['organisation_id', 'organisation_source'], how='left', indicator=True)

In [22]:
data_w_org[data_w_org['_merge'] == 'left_only']

,application_reference,award_reference,funder,administrator,application_source,start_date,end_date,funded_amount,application_title,decision_owner,application_category,lead_organisation_id,organisation_source,organisation_id,ukprn,organisation_name,_merge


In [23]:
data_w_org[data_w_org['organisation_name'] == 'University College London']

,application_reference,award_reference,funder,administrator,application_source,start_date,end_date,funded_amount,application_title,decision_owner,application_category,lead_organisation_id,organisation_source,organisation_id,ukprn,organisation_name,_merge
17,10672,MR/Z504713/1,MRC,MRC,TFS,2024-07-01,2029-06-30,2206878.69,Advancing lung function mapping with innovativ...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both
24,10787,MR/Z504129/1,MRC,MRC,TFS,2024-09-01,2027-08-31,411056.72,Studying the impact of progranulin mutations o...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both
33,11016,MR/Z504403/1,MRC,MRC,TFS,2024-09-01,2027-08-31,444439.32,Extending the genetic understanding of Thrombo...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both
46,11330,MR/Z504476/1,MRC,MRC,TFS,2024-05-01,2027-04-30,728765.66,Determining the causal features of hippocampal...,MRC,Research Grant,279,Siebel,279,10007784,University College London,both
52,11447,MR/Z504312/1,Unknown,MRC,TFS,2024-07-01,2028-06-30,1283465.48,Visualising the early secretory pathway in situ,MRC,Research Grant,279,Siebel,279,10007784,University College London,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207599,70,70,RE,RE,None,2020-04-01,2021-03-31,30000000.00,Project Oriel,RE,Capital grant,10007784,Unknown,10007784,None,University College London,both
207630,391,391,RE,RE,None,2020-04-01,2023-10-01,3953054.00,Transforming Academic-Policy Engagement (now c...,RE,None,10007784,Unknown,10007784,None,University College London,both
207672,496,496,RE,RE,None,2019-08-01,2020-08-01,509300.00,East London Inclusive Enterprise Zone,RE,None,10007784,Unknown,10007784,None,University College London,both
207753,843,843,RE,RE,None,2020-07-01,2022-06-01,414377.50,Increasing,RE,None,10007784,Unknown,10007784,None,University College London,both


In [24]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2024-09-17


In [25]:
print(data_w_org['funded_amount'].isna().sum())

print(len(data_w_org))


26609
207819


In [26]:
26609/207819

0.1280393034323137

In [27]:
print(data_w_org['start_date'].isna().sum())

print(len(data_w_org))


3137
207819


In [28]:

print(data_w_org[data_w_org['start_date'].isna()]['funded_amount'].sum())

# exclude those with start date is nan
data_ = data_w_org[~data_w_org['start_date'].isna()]


3626503.0


In [29]:


data_[data_['end_date'].isna()].sort_values(by='start_date')
# for end data is na put the start date

data_['end_date'] = pd.to_datetime(data_['end_date'].fillna(data_['start_date']))

# start date in the past
data_[data_['start_date'] < pd.to_datetime(today)]

# exclude those with start date greater than end date because we cannot say if they are active
data_ = data_[data_['start_date'] <= data_['end_date']]

print(data_['funded_amount'].isna().sum())

data_.drop(columns='funder', inplace=True)

data_['Active'] = 'Active'

data_.loc[data_.start_date > pd.to_datetime(today), 'Active'] = 'Future'

data_.loc[data_.end_date < pd.to_datetime(today), 'Active'] = 'Past'


C:\Users\MHil02\AppData\Local\Temp\ipykernel_5888\369158640.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_['end_date'] = pd.to_datetime(data_['end_date'].fillna(data_['start_date']))


23572


In [30]:
data_[data_['organisation_name'] == 'University College London']

,application_reference,award_reference,administrator,application_source,start_date,end_date,funded_amount,application_title,decision_owner,application_category,lead_organisation_id,organisation_source,organisation_id,ukprn,organisation_name,_merge,Active
17,10672,MR/Z504713/1,MRC,TFS,2024-07-01,2029-06-30,2206878.69,Advancing lung function mapping with innovativ...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both,Active
24,10787,MR/Z504129/1,MRC,TFS,2024-09-01,2027-08-31,411056.72,Studying the impact of progranulin mutations o...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both,Active
33,11016,MR/Z504403/1,MRC,TFS,2024-09-01,2027-08-31,444439.32,Extending the genetic understanding of Thrombo...,MRC,Fellowship,279,Siebel,279,10007784,University College London,both,Active
46,11330,MR/Z504476/1,MRC,TFS,2024-05-01,2027-04-30,728765.66,Determining the causal features of hippocampal...,MRC,Research Grant,279,Siebel,279,10007784,University College London,both,Active
52,11447,MR/Z504312/1,MRC,TFS,2024-07-01,2028-06-30,1283465.48,Visualising the early secretory pathway in situ,MRC,Research Grant,279,Siebel,279,10007784,University College London,both,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207599,70,70,RE,None,2020-04-01,2021-03-31,30000000.00,Project Oriel,RE,Capital grant,10007784,Unknown,10007784,None,University College London,both,Past
207630,391,391,RE,None,2020-04-01,2023-10-01,3953054.00,Transforming Academic-Policy Engagement (now c...,RE,None,10007784,Unknown,10007784,None,University College London,both,Past
207672,496,496,RE,None,2019-08-01,2020-08-01,509300.00,East London Inclusive Enterprise Zone,RE,None,10007784,Unknown,10007784,None,University College London,both,Past
207753,843,843,RE,None,2020-07-01,2022-06-01,414377.50,Increasing,RE,None,10007784,Unknown,10007784,None,University College London,both,Past


In [31]:
data_[data_['Active'] == 'Active'].sort_values(by='funded_amount', ascending=False).to_csv('active_grants.csv', index=False)

In [32]:
data_.drop(columns=['award_reference', 'application_source', 'lead_organisation_id', 'organisation_source', '_merge'], inplace=True)

In [33]:
writer = pd.ExcelWriter('active_grants.xlsx', engine = 'xlsxwriter')

In [34]:
data_[data_['Active'] == 'Active'].sort_values(by='funded_amount', ascending=False).to_excel(writer, sheet_name='Active', index=False)

In [35]:
data_[data_['Active'] == 'Future'].sort_values(by='funded_amount', ascending=False).to_excel(writer, sheet_name='Future', index=False)

In [36]:
df[df['start_date'].isna()].sum(numeric_only=True)

funded_amount    3626503.0
dtype: float64

In [37]:
# STFC grants pivot table with organisation, number of grants and total value

data_stfc = data_.copy()[(data_['Active'] != 'Past') & (data_['administrator'] == 'STFC')]
data_stfc['application_reference'].duplicated().sum()

data_select = data_stfc.copy()[['application_reference', 'start_date', 'funded_amount', 'organisation_name', 'Active']]

data_select['Year'] = data_select['start_date'].dt.year

data_Agg = data_select.groupby(['organisation_name', 'Active', 'Year'], as_index=False).agg({'application_reference':'nunique',
                                                                                  'funded_amount':'sum'})

In [38]:
data_Agg[data_Agg['Active'] == 'Active'].pivot(columns='Year', index='organisation_name', values='funded_amount').to_excel(writer, sheet_name='STFC Active Amount')

In [39]:
data_Agg[data_Agg['Active'] == 'Active'].pivot(columns='Year', index='organisation_name', values='application_reference').to_excel(writer, sheet_name='STFC Active Num')

In [40]:
data_Agg[data_Agg['Active'] == 'Future'].pivot(columns='Year', index='organisation_name', values='funded_amount').to_excel(writer, sheet_name='STFC Future Amount')

In [41]:
data_Agg[data_Agg['Active'] == 'Future'].pivot(columns='Year', index='organisation_name', values='application_reference').to_excel(writer, sheet_name='STFC Future Num')

In [42]:
writer.close()

In [43]:
writer = pd.ExcelWriter('Council_grants.xlsx', engine = 'xlsxwriter')

In [44]:
# grants pivot table with organisation, number of grants and total value

data_current = data_.copy()[(data_['Active'] != 'Past')]

data_select = data_current.copy()[['application_reference', 'administrator', 'start_date', 'funded_amount', 
                                   'organisation_name', 'Active']]

data_select['Year'] = data_select['start_date'].dt.year

data_Agg = data_select.groupby(['organisation_name', 'administrator', 'Active', 
                                'Year'], as_index=False).agg({'application_reference':'nunique',
                                                                'funded_amount':'sum'})

In [45]:
data_Agg[data_Agg['organisation_name'] == 0]

,organisation_name,administrator,Active,Year,application_reference,funded_amount


In [46]:
import numpy as np

In [47]:
data_Agg['funded_amount'].replace(0, np.nan, inplace=True)

data_active = data_Agg.copy()[data_Agg['Active'] == 'Active']

data_pivot = data_active.pivot(columns='Year', index=['organisation_name', 'administrator'], values='funded_amount').reset_index()

data_pivot.dropna(how='all', axis=1).to_excel(writer, sheet_name='Active Value', index=False)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_5888\2077863735.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_Agg['funded_amount'].replace(0, np.nan, inplace=True)


In [48]:
data_pivot = data_active.pivot(columns='Year', index=['organisation_name', 'administrator'], values='application_reference').reset_index()

data_pivot.dropna(how='all', axis=1).to_excel(writer, sheet_name='Active Number', index=False)

In [49]:
data_future = data_Agg.copy()[data_Agg['Active'] == 'Future']

data_pivot = data_future.pivot(columns='Year', index=['organisation_name', 'administrator'], values='application_reference').reset_index()

data_pivot.dropna(how='all', axis=1).to_excel(writer, sheet_name='Future Number', index=False)

In [50]:
data_pivot = data_future.pivot(columns='Year', index=['organisation_name', 'administrator'], values='funded_amount').reset_index()

data_pivot.dropna(how='all', axis=1).to_excel(writer, sheet_name='Future Amount', index=False)

In [51]:
writer.close()